# ★ 주식 단타  전략 모니터링 시스템 ★

[블로그 바로가기][URL]

[URL]: https://kirion.tistory.com/entry/%EC%A3%BC%EC%8B%9D-%EB%8B%A8%ED%83%80-%EC%A0%84%EB%9E%B5-%EB%AA%A8%EB%8B%88%ED%84%B0%EB%A7%81-%EC%8B%9C%EC%8A%A4%ED%85%9C-%EA%B0%9C%EB%B0%9C-%EC%9D%BC%EC%A7%80

# 0. 주식 단타 전략 모니터링 시스템 - 개요

주식 투자 ETF로 시작하라 라는 책에서 단타 전략 하나가 소개되었다.
래리 윌리엄스가 개발한 단기 기술적 지표로 단기적인 트레이딩을 하는 방법이다.   
   
전략은 다음과 같다.   
- 일봉 기준 Range 계산 : 전일 고가 - 저가   
- 매수 기준 : 당일 장중 가격 > 당일 시가 + Range -> 돌파 시점에 시장가 매수   
- 매도 기준 : 다음 날 시가 청산   

백테스팅 결과 지속적인 수익을 낼 수 있는 것으로 책에 쓰여 있고, 잘 활용하면 수익을 낼 수 있겠다 싶어 시스템을 개발하고자 한다.
   
본 시스템의 개발은 Python을 활용했으며 시스템의 필요한 구성 요소를 나열하면 아래와 같다.
   
- Python으로 주식 가격 조회   
- 매수 기준에 도달 감지   
- 카카오톡 API 나에게 메시지 보내기   
- 자동화 시스템 구현   
   
자동화 시스템을 만들려고 하는 것은 사람의 힘으로 모든 종목을 실시간으로 볼 수 없기 때문에 실시간 감지 시스템을 만들어 대상이 되는 종목을 탐지하고, 나에게 카카오톡으로 메시지로 매수 사인을 보내는 것이 목적이다.
   
또한, 날짜가 바뀌어도 시스템이 필요한 요소를 업데이트하고 지속 감지를 할 수 있게 구현이 목적이다.   
중간에 에러가 나 멈추더라도 카카오톡 알림을 통해 대응할 수 있게 한다.

# 1. 주식 단타 전략 모니터링 시스템 - 파이썬 주식 가격 조회

시스템을 개발하려면 먼저 주식 가격을 실시간으로 조회할 수 있어야 한다.
   
우선 python으로 주식 가격을 조회할 수 있는 방법을 소개한다.
   
python 패키지 중에 FinanceDataReader라는게 있다.
   
다른 패키지도 있는 것으로 아는데 개인의 사용 용도에 맞춰 사용하면 될 것 같다.
   
   
python과 해당 패키지가 설치되어 있다 보고, 코드 실행 결과로 간단하게 설명하겠다.
자세한 내용은 패키디 Documentation을 참조 바란다.
   
   
개발 환경 : python 3.8, Jupyter Notebook

**<종목 코드>**   
- 거래소별 전체 종목코드: KRX (KOSPI, KODAQ, KONEX), NASDAQ, NYSE, AMEX, S&P 500
   
**<가격 데이터>**   
- 해외주식 가격 데이터: AAPL(애플), AMZN(아마존), GOOG(구글) 등   
- 국내주식 가격 데이터: 005930(삼성전자), 091990(셀트리온헬스케어) 등   
- 각종 지수: KS11(코스피지수), KQ11(코스닥지수), DJI(다우지수), IXIC(나스닥 지수), US500(S&P 5000)   
- 환율 데이터: USD/KRX (원달러 환율), USD/EUR(달러당 유로화 환율), CNY/KRW: 위엔화 원화 환율   
- 암호화폐 가격: BTC/USD (비트코인 달러 가격, Bitfinex), BTC/KRW (비트코인 원화 가격, 빗썸)   

In [32]:
import FinanceDataReader as fdr
#df_krx = fdr.StockListing('KRX')
# sector 정보를 구하기 위해 'KRX' 대신 'KOSPI-DESC' 사용
df_krx = fdr.StockListing('KOSPI-DESC')
df_krx.head(5)
#df_krx.to_csv('KRX_list_20240502.csv',encoding='cp949')  # 한글 정상 출력을 위해 encoding 방식은 'cp949'

,Code,Name,Market,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,095570,AJ네트웍스,KOSPI,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,손삼달,http://www.ajnet.co.kr,서울특별시
1,006840,AK홀딩스,KOSPI,기타 금융업,지주사업,1999-08-11,12월,"채형석, 백차현(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
2,027410,BGF,KOSPI,기타 금융업,지주회사,2014-05-19,12월,홍정국,http://www.bgf.co.kr,서울특별시
3,282330,BGF리테일,KOSPI,종합 소매업,체인화 편의점,2017-12-08,12월,민승배,http://www.bgfretail.com,서울특별시
4,138930,BNK금융지주,KOSPI,기타 금융업,금융지주회사,2011-03-30,12월,빈대인,http://www.bnkfg.com,부산광역시


[1] FinanceDataReader를 import 함.   
[2] StockListing을 통해 종목 리스트를 불러오는데 우리는 한국 주식 시장을 이용함으로 "KRX"를 입력함.   
[3] 불러오 리스트를 확인. KOSDAQ과 KOSPI 종목의 내역이 보인다.

In [33]:
kospi_list = df_krx[df_krx['Market'] == 'KOSPI']
kospi_list.head()

,Code,Name,Market,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,095570,AJ네트웍스,KOSPI,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,손삼달,http://www.ajnet.co.kr,서울특별시
1,006840,AK홀딩스,KOSPI,기타 금융업,지주사업,1999-08-11,12월,"채형석, 백차현(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
2,027410,BGF,KOSPI,기타 금융업,지주회사,2014-05-19,12월,홍정국,http://www.bgf.co.kr,서울특별시
3,282330,BGF리테일,KOSPI,종합 소매업,체인화 편의점,2017-12-08,12월,민승배,http://www.bgfretail.com,서울특별시
4,138930,BNK금융지주,KOSPI,기타 금융업,금융지주회사,2011-03-30,12월,빈대인,http://www.bnkfg.com,부산광역시


[4] 코스피 종목만 조회하기 위해 Market 컬럼에서 "KOSPI"를 입력.   
[5] 코스피 내역
   
   
그런데  ACE란 이름이 보인다. ETF이름으로 검색해 특정 기업의 ETF 목록을 뽑을 수 있다.

In [35]:
#df_krx[df_krx['Name'].str.contains('KODEX')].head()
# ETF는 아래와 같이 별도로 조회하여야 함
etf_list = fdr.StockListing('ETF/KR')
etf_list[etf_list['Name'].str.contains('KODEX')].head()

,Symbol,Category,Name,Price,RiseFall,Change,ChangeRate,NAV,EarningRate,Volume,Amount,MarCap
0,459580,6,KODEX CD금리액티브(합성),1034755,2,290,0.03,1034554.0,0.9056,231679,239731,85131
2,069500,1,KODEX 200,36585,5,-65,-0.18,36663.0,9.1228,1576016,57573,60329
3,423160,6,KODEX KOFR금리액티브(합성),106625,2,10,0.01,106617.0,0.8990,879169,93741,53841
7,273130,6,KODEX 종합채권(AA-이상)액티브,109355,2,170,0.16,109309.0,-0.0778,6978,762,28038
8,465680,6,KODEX 24-12 은행채(AA+이상)액티브,10275,3,0,0.00,10276.0,0.8836,16739,171,25858


[6] KODEX가 제공하는 ETF 목록을 조회하기 위해 KODEX 문자열을 포함하는 이름의 종목을 불러왔다.
   
여기서 개별 종목을 얻기를 원하기 때문에 한 번 더 필터를 걸어줬다.

In [36]:
kospi_list = kospi_list[~kospi_list['Sector'].isna()]
kospi_list.head()

,Code,Name,Market,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,095570,AJ네트웍스,KOSPI,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,손삼달,http://www.ajnet.co.kr,서울특별시
1,006840,AK홀딩스,KOSPI,기타 금융업,지주사업,1999-08-11,12월,"채형석, 백차현(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
2,027410,BGF,KOSPI,기타 금융업,지주회사,2014-05-19,12월,홍정국,http://www.bgf.co.kr,서울특별시
3,282330,BGF리테일,KOSPI,종합 소매업,체인화 편의점,2017-12-08,12월,민승배,http://www.bgfretail.com,서울특별시
4,138930,BNK금융지주,KOSPI,기타 금융업,금융지주회사,2011-03-30,12월,빈대인,http://www.bnkfg.com,부산광역시


7] ETF는 Sector나 Industry에 값이 NaN으로 공란이다. 따라서 공란이 아닌 대상으로 필터를 걸어 KOSPI 종목만 추려냈다.   
   
***※ 'KOSPI-DESC' 로 가져왔으므로, 상기 필터는 불필요***

In [37]:
kospi_list['Sector'].value_counts().head(10)

Sector
기타 금융업           65
자동차 신품 부품 제조업    45
의약품 제조업          41
1차 철강 제조업        39
기타 화학제품 제조업      32
기초 화학물질 제조업      28
부동산 임대 및 공급업     26
전자부품 제조업         23
플라스틱제품 제조업       19
금융 지원 서비스업       19
Name: count, dtype: int64

In [38]:
kospi_list.shape[0]

840

[9] Sector의 현황 및 불필요한 값이 있는지 확인한다. 캡쳐에서는 상위 10개를 나타냈다.   
[10] 총 823개의 KOSPI 종목 리스트를 만들었다.

In [51]:
from datetime import datetime

samsung = fdr.DataReader('005930',datetime.now().date())
samsung

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2024-05-02,77600,78600,77300,78100,15323237,0.007742


In [52]:
samsung = fdr.DataReader('005930', '2022')
samsung

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,79400,79800,78200,78600,13502112,0.003831
2022-01-04,78800,79200,78300,78700,12427416,0.001272
2022-01-05,78800,79000,76400,77400,25470640,-0.016518
2022-01-06,76700,77600,76600,76900,12931954,-0.006460
2022-01-07,78100,78400,77400,78300,15163757,0.018205
...,...,...,...,...,...,...
2024-04-25,77300,77500,76300,76300,15549134,-0.029262
2024-04-26,77800,77900,76500,76700,12755629,0.005242
2024-04-29,77400,77600,76200,76700,14664474,0.000000


[11] 종목의 주가 조회를 위해 DataReader 함수를 사용한다. 종목코드와 시점을 입력하면   
[12] 10월 19일 자의 시가, 고가, 저가, 종가, 거래량, 등락비율을 볼 수 있다.   
[13] 날짜가 아닌 기간으로도 확인이 가능하다. 2022를 입력하면 2022년의 모든 거래일의 주가를 뽑을 수 있다.

이 글을 작성하는 현재 10월 19일 15:10분 경이다. 아직 주식 시장이 열려있는 시간이기에 주가 조회가 실시간으로 변경되는지 확인해본다.   
[16] 앞서 10분 전에 조회한 결과인데 종가 부분을 뜻하는 Close가 55,900, Volume이 10,777,303으로 나타난다.   
[18] 다시 조회했을 땐 Close가 56,000, Volume이 11,064,172로 변동된 것을 볼 수 있다.   
즉, 주식 시장이 열려있을 때는 실시간으로 가격이 변동이 되는 것이다.   
   
하지만, 반복적으로 확인한 결과 주식 시장의 데이터가 반영되는 데는 1~2분 정도 차이가 존재했다.   
시스템 개발에 오차 없이 실시간으로 감지해야 하면 이 오차는 치명적일 수 있기에 이런 점을 고려하는 것이 필수이겠다.   
   
이렇게 종목 리스트와 주가를 확보했고, 다음으로는 매수 시점을 감지하는 코드를 구현해 보겠다.

# 2. 주식 단타 전략 모니터링 시스템 - 매수 기준 감지

두 번째 글은 단기 기술적 지표를 이용해 매수 종목 감지를 주제로 한다.   
   
첫 글에서 소개했듯이 래리 윌리엄스가 개발한 단기 매매 방법으로 다시 한번 설명하면   
   
단기 매매 전략   
- 일봉 기준 Range 계산 : 전일 고가 - 저가   
- 매수 기준 : 당일 장중 가격 > 당일 시가 + Range -> 돌파 시점에 시장가 매수   
- 매도 기준 : 다음 날 시가 청산   
   
앞선 글에선 매매 기준을 찾기 위해 FinanceDataReader 패키지로 주식 종목 및 주가를 수집했다.

In [53]:
import FinanceDataReader as fdr
import pandas as pd

#df_krx = fdr.StockListing('KRX')
# sector 정보를 구하기 위해 'KRX' 대신 'KOSPI-DESC' 사용
df_krx = fdr.StockListing('KOSPI-DESC')
df_krx.head(5)

,Code,Name,Market,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,095570,AJ네트웍스,KOSPI,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,손삼달,http://www.ajnet.co.kr,서울특별시
1,006840,AK홀딩스,KOSPI,기타 금융업,지주사업,1999-08-11,12월,"채형석, 백차현(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
2,027410,BGF,KOSPI,기타 금융업,지주회사,2014-05-19,12월,홍정국,http://www.bgf.co.kr,서울특별시
3,282330,BGF리테일,KOSPI,종합 소매업,체인화 편의점,2017-12-08,12월,민승배,http://www.bgfretail.com,서울특별시
4,138930,BNK금융지주,KOSPI,기타 금융업,금융지주회사,2011-03-30,12월,빈대인,http://www.bnkfg.com,부산광역시


[1] ~ [3] 주가 종목을 수집.

In [55]:
kospi_data = df_krx[df_krx['Market'].str.contains('KOSPI')]
kopsi_data = kospi_data[~kospi_data['Sector'].isna()]  # ETF종목은 안 가져오므로 필요 없음

[4] KOSPI 데이터 필터   
[5] ETF 종목 제외 필터

In [56]:
kospi_200_list = pd.read_csv('./KOSPI200.csv', index_col = 0, encoding='cp949')
list_200 = kospi_200_list[kospi_200_list['Symbol']==102110].STK_NM_KOR
list_200

0          삼성전자
1        SK하이닉스
2      LG에너지솔루션
3      삼성바이오로직스
4           현대차
         ...   
195        KG스틸
196        신풍제약
197        명신산업
198          영풍
199      PI첨단소재
Name: STK_NM_KOR, Length: 200, dtype: object

[6] ~ [8] KOSPI 대표종목 200개를 얻기 위해 Tiger etf 200 구성 종목을 가져왔다. 해당 파일은 내가 200개 구성을 추리기 위해 따로 만들 파일이다.   
따라서 동일하게 하지 않고 사용자가 원하는 데로 종목 리스트를 구성해서 사용할 수 있다.   
예를 들어 시가총액 상위 20개 기업 대상, 코스닥 기업 대상, 특정 섹터 대상 등 원하는 종목을 구성하면 된다.

In [57]:
kospi_data = kospi_data[kospi_data['Name'].isin(list_200)]
kospi_data.shape[0]

200

[9] ~ [10] 200개 기업을 추렸는데 199개가 나왔다. (9월 말 기준으로 추린 건데 한 개 기업이 부도가 났나..)

In [63]:
t_date = '2024-04-30'  #전일자

In [64]:
before_price = None
for t_code in kospi_data['Code']:
    t_name = kospi_data[kospi_data['Code']==t_code]["Name"].values[0]
    price_data = fdr.DataReader(t_code, t_date, t_date)
    price_data.insert(0, 'Code', t_code)
    price_data.insert(0, 'Name', t_name)
    before_price = pd.concat([before_price, price_data], axis = 0)
before_price['band'] = before_price['High'] - before_price['Low']
before_price.head(5)

,Name,Code,Open,High,Low,Close,Volume,Change,band
Date,,,,,,,,,
2024-04-30,BGF리테일,282330,130200,132900,127600,131100,78005,0.008462,5300
2024-04-30,BNK금융지주,138930,8370,8470,8200,8390,1407089,-0.009445,270
2024-04-30,CJ,001040,127800,131300,126500,127700,156613,0.000000,4800
2024-04-30,CJ대한통운,000120,123100,124200,122000,122600,64460,-0.012882,2200
2024-04-30,CJ제일제당,097950,335500,342500,335500,337000,42984,0.007474,7000


[11] 작성하는 날이 2022년 10월 19일이므로 하루 전날인 18일의 주가를 수집.   
[12] 199개 기업의 종목 코드를 이용해 하나씩 수집한다.   
[13] 수집된 18일 기준의 주가정보   
앞선 데이터와 다르게 band 컬럼이 생성이 되었는데 이는 매수 기준가를 만들기 위한 값으로 고가 - 저가이다.

In [65]:
t_date = '2024-05-01'  #현재일자
t_date = datetime.now().date()
print(t_date)

2024-05-02


In [66]:
%%time
for t_code in kospi_data['Code']:
    t_name = kospi_data[kospi_data['Code']==t_code]["Name"].values[0]
    price_data = fdr.DataReader(t_code, t_date)
    
    target_band = before_price[before_price['Code'] == t_code]['band'].values[0]
    price_data['Base_Price'] = price_data['Open'] + int(target_band * 1.5)
    
    base_price = price_data['Base_Price'].values[0]
    current_price = price_data['Close'].values[0]
    
    if current_price >= base_price and target_band >= price_data['Open'].values[0] * 0.005:
        print(t_name, '현재 가격 : ', current_price, '기준 가격 : ', base_price)
    
print('Done')

CJ제일제당 현재 가격 :  345000 기준 가격 :  345000
DB하이텍 현재 가격 :  42700 기준 가격 :  42500
DL 현재 가격 :  58000 기준 가격 :  54050
F&F 현재 가격 :  69900 기준 가격 :  68450
KT&G 현재 가격 :  91100 기준 가격 :  90200
금호타이어 현재 가격 :  7830 기준 가격 :  7655
신풍제약 현재 가격 :  14930 기준 가격 :  14585
오뚜기 현재 가격 :  426000 기준 가격 :  424000
한온시스템 현재 가격 :  5870 기준 가격 :  5805
현대글로비스 현재 가격 :  184500 기준 가격 :  181650
Done
CPU times: total: 5min 29s
Wall time: 6min 33s


[14] 오늘 가격을 조회해야하기 때문에 10월 19일로 설정   
[15] for문을 사용해 기업 별 현재 가격을 조회해 매수 기준에 충족하는지 감지한다.   
매수 기준가는 당일 시가 + band인데 단기 기술적 지표로 사용하기 위한 기본값은 1.5배를 곱한 값을 사용한다고 한다.   
따라서 기준과 동일하게 1.5배를 곱해서 더해 주었고, 매수 기준가(base_price)를 설정한다.   
이제 현재가를 비교하기 위해 Close열을 사용하는데, 1~2분 차이나는 현재가 혹은 시장이 끝났을 때는 종가이기에 비교 값으로 사용했다.   
   
조건문은 현재가가 매수 기준가보다 크면 매수 사인을 보내는 것인데, band값이 매우 작은 경우나 0인 경우가 간혹 있어서 band가 시가의 0.5% 가격을 넘는 기준을 추가했다.   
두 번째 조건은 선택적으로 사용하면 될 것 같다.   
   
내가 선택한 199개 기업에 대해서 매수 기준을 통과하는 기업을 뽑아봤는데 1개 기업도 뽑히지 않았다....   
오늘 장이 약세라 그런가..

In [67]:
%%time
for t_code in kospi_data['Code']:
    t_name = kospi_data[kospi_data['Code']==t_code]["Name"].values[0]
    price_data = fdr.DataReader(t_code, t_date)
    
    target_band = before_price[before_price['Code'] == t_code]['band'].values[0]
    price_data['Base_Price'] = price_data['Open'] + int(target_band * 1.5)
    
    base_price = price_data['Base_Price'].values[0]
    current_price = price_data['High'].values[0]
    
    if current_price >= base_price and target_band >= price_data['Open'].values[0] * 0.005:
        print(t_name, '현재 가격 : ', current_price, '기준 가격 : ', base_price)
    
print('Done')

CJ제일제당 현재 가격 :  345500 기준 가격 :  345000
DB하이텍 현재 가격 :  42700 기준 가격 :  42500
DL 현재 가격 :  58500 기준 가격 :  54050
F&F 현재 가격 :  70300 기준 가격 :  68450
KT&G 현재 가격 :  91200 기준 가격 :  90200
LIG넥스원 현재 가격 :  180600 기준 가격 :  179000
금호타이어 현재 가격 :  7900 기준 가격 :  7655
녹십자 현재 가격 :  116500 기준 가격 :  115900
농심 현재 가격 :  410000 기준 가격 :  407000
대한유화 현재 가격 :  144000 기준 가격 :  143850
두산퓨얼셀 현재 가격 :  21000 기준 가격 :  20790
신풍제약 현재 가격 :  15300 기준 가격 :  14585
에스원 현재 가격 :  60900 기준 가격 :  60800
엔씨소프트 현재 가격 :  179700 기준 가격 :  179400
오뚜기 현재 가격 :  430000 기준 가격 :  424000
일진하이솔루스 현재 가격 :  24800 기준 가격 :  24275
풍산 현재 가격 :  75000 기준 가격 :  74250
한국앤컴퍼니 현재 가격 :  17650 기준 가격 :  17630
한온시스템 현재 가격 :  5930 기준 가격 :  5805
현대글로비스 현재 가격 :  184800 기준 가격 :  181650
Done
CPU times: total: 5min 1s
Wall time: 5min 32s


[16] 고가 기준으로 다시 감지해봤다.   
Close에서 High로 바꾸면 해당 날짜의 고가를 사용해 비교할 수 있다.   
이렇게 조회해보니 7개 기업이 감지 망에 걸렸다!   
감지 망에 걸린 후 종가는 모든 기업이 기준 가격에 낮아져서 안 나왔기에 전략적인 접근이 필요해 보이지만,   
일단은 매수 시점 감지를 하는 기초적인 프로그래밍은 구현을 했다.   
   
다음으로는 단기 기술적 지표로 과거 자료로 백테스팅을 했을 때 수익률이 어느 정도를 낼 수 있는지 검증해 보겠다.

# 3. 주식 단타 전략 모니터링 시스템 - 단기 기술적 지표 백테스팅

앞 글에서 래리 윌리엄스의 전략을 간단하게 구현해봤는데 과연 이 전력이 잘 먹힐까 라는 의문을 갖게 되었다.   
다른 포스팅 글에서 충분히 검증되었지만, 내가 직접 구현해서 검증하는 것과는 다르기 때문에 Python으로 직접 구현해보았다.   
기준은 전일 고가 - 저가로 계산한 Range의 1.5배를 적용하고, 매도 수수료 0.1%를 적용했다.   
입력 인수로 배수와 수수료를 바뀌가며 테스트할 수 있게 구현하였다.

In [68]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np

df_krx = fdr.StockListing('KRX')
# kospi_data = df_krx[(df_krx['Market']=='KOSPI') & (~df_krx['Sector'].isna())]
kospi_data = df_krx[(df_krx['Market']=='KOSPI')]
kospi_data.shape

(953, 17)

먼저 동일하게 주가 정보를 불러오고 KOSPI만 뽑는다.

In [69]:
kospi_price_data = None
for t_i in kospi_data.Code:
    t_n = kospi_data[kospi_data['Code']==t_i]['Name'].values[0]
    price_data = fdr.DataReader(t_i, '2019-01-01')
    price_data.insert(0,'Code', t_i)
    price_data.insert(0,'Name', t_n)
    
    kospi_price_data = pd.concat([kospi_price_data, price_data], axis = 0)

In [71]:
#kospi_price_data.to_csv('./kospi_data.csv', encoding='cp949')
kospi_price_data = pd.read_csv('./kospi_data.csv', index_col=0, dtype={'Code':str}, encoding='cp949')

주가를 수집하기 위해 823개 기업에 대한 주가를 모두 수집한다.   
나는 2019년부터 현재까지의 데이터를 수집했다.   
시간이 꽤 소요되기 때문에 한 번 수집 후 저장해놓았다.   
수집한 데이터를 불러오고

In [72]:
kospi_price_data['band'] = kospi_price_data['High'] - kospi_price_data["Low"]

Range를 미리 계산해 놓는다.

In [73]:
def trading_simul(tmp, per, tax, pr):
    my_money = 1000000
    before_day = None
    current_day = None
    buy = False
    sell = False
    for i in tmp.index:
        if before_day is None:
            before_day = tmp[tmp.index == i]
        else:
            current_day = tmp[tmp.index == i]
            if buy == True:
                sell_price = current_day['Open'].values[0]
                sell_money = sell_price * stock_num * tax
                my_money = my_money + sell_money
                diff = sell_price - buy_price
                diff_per = np.round(diff/buy_price*100, 2)
                
                print(i, "매수가 :", buy_price, "/ 매도가 :", sell_price, "/ 보유금액 :", my_money, "/ 손익률 :", diff_per)
                buy = False
            base_price = current_day['Open'].values[0] + int((before_day['band'].values[0] *per))
            
            if current_day['High'].values[0] >= base_price and before_day['band'].values[0] >= current_day['Open'].values[0]*0.005:
                buy = True
                if pr == 'Close':
                    buy_price = current_day['Close'].values[0]
                elif pr == 'Base':
                    buy_price = base_price
                stock_num = int(my_money/buy_price)
                buy_money = buy_price * stock_num
                my_money = my_money - buy_money
                
            before_day = current_day.copy()

시뮬레이션을 하기 위한 함수이다.   
   
tmp에는 한 개 기업의 2019년 부터 현재까지 주가를 받도록 했다.   
per는 Range의 몇 배를 이용할 지 입력받는다.(default :1.5)   
tax는 매도 수수료 (default :0.999)   
pr은 종가로 할지, 매수 기준가로 할지 선택한다(default : "Base", or "Close)   
시간 순으로 계산이 되며 전날 Range를 이용해 당일의 매수 기준가를 계산한 후 당일 가격이 넘으면 매수한다.   
   
한 가지, 이 데이터는 과거 데이터라 시가, 고가, 저가, 종가 밖에 없다.   
따라서 어떤 가격으로 매수 시뮬레이션을 할지 선택해야 한다.   
매수 기준가를 돌파하는 순간 매수할지, 종가에 매수할지 선택할 수 있도록 구현했다.

In [74]:
kospi_price_data.sample(10)

,Name,Code,Open,High,Low,Close,Volume,Change,band
Date,,,,,,,,,
2021-02-09,일동홀딩스,000230,16400,16600,16000,16050,53505,-0.021341,600
2021-03-24,ESR켄달스퀘어리츠,365550,5794,5902,5774,5873,360012,0.011888,128
2022-08-11,현대코퍼레이션홀딩스,227840,11500,11600,11400,11550,12545,0.004348,200
2020-09-18,메타랩스,090370,5900,5900,5625,5875,75590,-0.004237,275
2019-07-15,삼성물산우B,02826K,69500,69700,69100,69500,1267,-0.002869,600
2021-09-06,LG,003550,95100,96300,94600,95300,146255,-0.011411,1700
2021-12-15,금호건설,002990,11950,12350,11950,12100,93359,0.000000,400
2024-02-05,제이준코스메틱,025620,5770,5900,5730,5760,10089,-0.003460,170
2020-03-09,한세실업,105630,13300,13300,12250,12550,264400,-0.073801,1050


수집한 기업 중 한 개 기업을 선택한다.

In [75]:
tmp = kospi_price_data[kospi_price_data['Code'] == '000270']

기아로 해보자.

In [76]:
trading_simul(tmp, 1.5, 0.999, 'Base')

2019-02-21 매수가 : 36500 / 매도가 : 35200 / 보유금액 : 963949.6 / 손익률 : -3.56
2019-03-19 매수가 : 34950 / 매도가 : 34450 / 보유금액 : 949519.45 / 손익률 : -1.43
2019-04-01 매수가 : 35650 / 매도가 : 35500 / 보유금액 : 944696.45 / 손익률 : -0.42
2019-04-04 매수가 : 35975 / 매도가 : 36500 / 보유금액 : 957397.45 / 손익률 : 1.46
2019-04-09 매수가 : 38125 / 매도가 : 39050 / 보유금액 : 979546.2 / 손익률 : 2.43
2019-04-11 매수가 : 40025 / 매도가 : 40150 / 보유금액 : 981582.6 / 손익률 : 0.31
2019-04-17 매수가 : 41850 / 매도가 : 42050 / 보유금액 : 985215.45 / 손익률 : 0.48
2019-04-19 매수가 : 43400 / 매도가 : 43000 / 보유금액 : 975469.45 / 손익률 : -0.92
2019-04-30 매수가 : 45100 / 매도가 : 45350 / 보유금액 : 979767.1 / 손익률 : 0.55
2019-06-25 매수가 : 43525 / 매도가 : 43150 / 보유금액 : 970567.7999999999 / 손익률 : -0.86
2019-08-27 매수가 : 42500 / 매도가 : 42250 / 보유금액 : 964138.2999999999 / 손익률 : -0.59
2019-09-03 매수가 : 44050 / 매도가 : 44000 / 보유금액 : 962164.2999999999 / 손익률 : -0.11
2019-09-24 매수가 : 45125 / 매도가 : 46450 / 보유금액 : 989013.85 / 손익률 : 2.94
2019-11-08 매수가 : 43275 / 매도가 : 43200 / 보유금액 : 986413.45 / 손익률 : -0.17
2019-1

매수 기준가로 시뮬레이션 결과 100만원으로 투자 시 157만원이 되었다. 결과가 길어 생략했지만, 57% 수익이 볼 수 있다.   
2019년부터 3년 9개월 정도 백테스팅 결과이다.   
매수 기준가를 돌파하는 순간 제때만 사더라도 충분히 수익을 낼 수 있음을 확인했다.

In [77]:
trading_simul(tmp, 1.5, 0.999, 'Close')

2019-02-21 매수가 : 35200 / 매도가 : 35200 / 보유금액 : 999014.4 / 손익률 : 0.0
2019-03-19 매수가 : 34700 / 매도가 : 34450 / 보유금액 : 991049.8 / 손익률 : -0.72
2019-04-01 매수가 : 35350 / 매도가 : 35500 / 보유금액 : 994255.8 / 손익률 : 0.42
2019-04-04 매수가 : 36400 / 매도가 : 36500 / 보유금액 : 995970.3 / 손익률 : 0.27
2019-04-09 매수가 : 39000 / 매도가 : 39050 / 보유금액 : 996244.05 / 손익률 : 0.13
2019-04-11 매수가 : 40100 / 매도가 : 40150 / 보유금액 : 996480.4500000001 / 손익률 : 0.12
2019-04-17 매수가 : 41800 / 매도가 : 42050 / 보유금액 : 1001263.3 / 손익률 : 0.6
2019-04-19 매수가 : 42550 / 매도가 : 43000 / 보유금액 : 1010624.3 / 손익률 : 1.06
2019-04-30 매수가 : 45250 / 매도가 : 45350 / 보유금액 : 1011826.6000000001 / 손익률 : 0.22
2019-06-25 매수가 : 43400 / 매도가 : 43150 / 보유금액 : 1005084.1500000001 / 손익률 : -0.58
2019-08-27 매수가 : 42100 / 매도가 : 42250 / 보유금액 : 1007562.4000000001 / 손익률 : 0.36
2019-09-03 매수가 : 43900 / 매도가 : 44000 / 보유금액 : 1008794.4000000001 / 손익률 : 0.23
2019-09-24 매수가 : 46400 / 매도가 : 46450 / 보유금액 : 1008868.9500000002 / 손익률 : 0.11
2019-11-08 매수가 : 43250 / 매도가 : 43200 / 보유금액 : 1006725.

매수 기준가 시점을 놓쳐 종가로 샀다고 가정했을 땐 142만원으로 42% 수익이 난다.

**결론**   
단기 기술적 지표로 충분히 수익을 낼 수 있다는 것을 확인했다.   
물론 매도 수수료를 각 증권사 별 정확한 수치를 넣어서 봐야 하지만, 가능성을 확인한 것에 의의를 둔다.   
하지만 종목에 따라 편차가 심해 종목 선택을 잘해야 이 정도 수익을 낼 가능성이 있어 보인다.   
등락이 심한 종목을 선택할 경우 제때 매수를 할 수 없을 뿐더러 수익률이 마이너스로 가는 경우도 나타났기 때문이다.

# 4. 주식 단타 전략 모니터링 시스템 - 카카오톡 API 나에게 메세지 보내기

앞서 단기 기술적 지표로 수익이 단기 매매의 가능성을 확인해보았다.   
   
그럼 이제 실시간으로 어떤 종목이 기준 지표를 돌파하는지 감지를 해야 하는데   
내가 하나하나 지켜볼 수 없기 때문에 돌파하는 종목을 카카오톡으로 메세지를 보내 알려주려 한다.   
   
카카오톡 API를 사용하기 위해선 카카오톡 개발자에 가입해야 한다.   
   
간략히 순서를 말하자면   
application 생성 - rest api 값 확인 - 카카오 로그인 설정 및 메세지 보내기 허용 - code 값 확인 - 인증 token 발급이다.   
   
하나하나 보면서 진행해보자.   
   
우선 카카오 developers 가입   
   
https://developers.kakao.com/ 

code값은   
https://kauth.kakao.com/oauth/authorize?client_id=aa96073b402b59689b2e3bb12c1390dc&redirect_uri=https://www.naver.com&response_type=code 
   
해당 코드값이 나오므로 copy하여 사용

In [23]:
import requests
import json

# 카카오톡 메시지 API
url = "https://kauth.kakao.com/oauth/token"

data = {
    "grant_type":"authorization_code",
    "client_id":"aa96073b402b59689b2e3bb12c1390dc",
    "redirect_url":"https://www.naver.com",
    "code":"TX0P5GKRAxGaqgqe-VBfl8AHbkl00JPJZhnw1PchnxoirOrHu_fyBKJlt50KKiURAAABjzwobAjUNEQ5evY1pg",
}
response = requests.post(url, data=data)
tokens = response.json()
print(tokens)

{'access_token': 'ya0DdmvPtlFpWDTPEhqFKUGRG1puzGqlTbsKPXVcAAABjzworwH7Ewsnpgvovw', 'token_type': 'bearer', 'refresh_token': 'SQItPIQeeCN0XrWmnbzhSCsstobItRQlhCUKPXVcAAABjzworv77Ewsnpgvovw', 'expires_in': 21599, 'scope': 'talk_message', 'refresh_token_expires_in': 5183999}


저장한 키값과 code값을 이용해 토큰을 발행한다.   
밑줄 친 부분(client_id, redirect_url, code)을 수정하면 된다.

In [24]:
# kakao_code.json 파일 저장
with open("kakao_code.json", "w") as fp:
    json.dump(tokens, fp)

# kakao_code.json 파일 불러오기
with open("kakao_code.json", "r") as fp:
    tokens = json.load(fp)

# 토큰 재발행
def refreshToken(refresh_token):
    REST_API_KEY = "aa96073b402b59689b2e3bb12c1390dc"
    REDIRECT_URI = "https://www.naver.com"
    data = {
        "grant_type": "refresh_token",
        "client_id":f"{REST_API_KEY}",
        "refresh_token": refresh_token
    }
    
    resp = requests.post(REDIRECT_URI , data=data)
    print(resp)
    new_token = resp.json()
    
    return new_token['access_token']

카카오톡 API 토큰은 사용기간이 짧기 때문에 주기적으로 재발행해주어야 한다.   
재발행을 위한 코드는 위의 함수로 구현하였다.   
재발행 함수도 밑줄 친 부분을 수정하면 된다.

In [86]:
# 메세지 보내기
url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

headers = {
    "Authorization": "Bearer " + tokens["access_token"]
}

# 보내고자 하는 data 입력
data = {"template_object" : json.dumps({ "object_type" : "text",
                                        "text" : "매수사인"+"\n" +
                                        "매수종목 : " + "삼성전자 " ,
                                        "link" : None  })
       } 

response = requests.post(url, headers=headers, data=data)

# 메세지 전송
if response.json().get('result_code') == 0:
    print('메시지를 성공적으로 보냈습니다.')
else:
    print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(response.json()))
    tokens["access_token"] = refreshToken(tokens["refresh_token"])
    headers = {"Authorization": "Bearer " + tokens["access_token"]}

# 메세지가 전송이 안되면 토큰의 기한이 다 되었다는 뜻이기 때문에 재발행을 받은 후 headers의 tokens값을 경신해준다.

메시지를 성공적으로 보냈습니다.


이렇게 하면 카카오톡으로 메세지 보내기를 할 수 있다.   
   
단점은 소리로 알람이 따로 오지 않는다는 점이다.   
   
이렇게 카카오톡 API로 나에게 알림 메세지를 주는 것까지 구현하였다.   
다음부터는 실시간으로 감지해 보내는 글을 작성하겠다.

# 5. 주식 단타 전략 모니터링 시스템 - 파이썬 자동 감지 프로그램(1)

주식 가격 조회, 매수 기준 감지, 카카오톡 API가 준비가 되었다면, 이제 자동 감지 프로그램을 구축할 차례다.

In [1]:
import FinanceDataReader as fdr
import numpy as np
import pandas as pd

개발에 필요한 패키지를 import 해주고

In [2]:
df_krx = fdr.StockListing('KRX')

kospi_data = df_krx[df_krx["Market"].str.contains("KOSPI")]
#kospi_data = kospi_data[~kospi_data["Sector"].isna()]

Kospi 종목만 가져온다.   
200개 종목만 가져오기 위해 Tiger 200에 포함되는 종목만 뽑았다.

In [3]:
kospi_200_list = pd.read_csv('./KOSPI200.csv', index_col = 0, encoding='cp949')
kospi_200_list[kospi_200_list['Symbol']==102110]

,Symbol,Code,STK_NM_KOR
0,102110,5930,삼성전자
1,102110,660,SK하이닉스
2,102110,373220,LG에너지솔루션
3,102110,207940,삼성바이오로직스
4,102110,5380,현대차
...,...,...,...
195,102110,16380,KG스틸
196,102110,19170,신풍제약
197,102110,9900,명신산업
198,102110,670,영풍


전체 종목 중 200개 기업으로 조건문을 걸어 뽑아낸다.

In [4]:
kospi_data = kospi_data[kospi_data["Name"].isin(kospi_200_list[kospi_200_list["Symbol"]==102110].STK_NM_KOR)]
kospi_data.head(10)

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,78400,1,400,0.51,79000,79000,78100,2610402,205276659600,468030951920000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,176300,1,2700,1.56,177000,177500,176100,510337,90254299200,128346816949500,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,391000,1,2000,0.51,391000,391500,389000,6687,2608720000,91494000000000,234000000,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,779000,1,4000,0.52,779000,781000,776000,4642,3614830000,55444546000000,71174000,STK
5,005380,KR7005380001,현대차,KOSPI,,249500,1,500,0.20,250000,250500,248000,49588,12354546500,52249339654500,209416191,STK
6,000270,KR7000270009,기아,KOSPI,,119500,2,-100,-0.08,120300,120800,118400,178823,21377210600,48044282258500,402044203,STK
7,068270,KR7068270008,셀트리온,KOSPI,,188800,1,1300,0.69,188900,189600,188200,30474,5755523800,41167795065600,218049762,STK
8,005490,KR7005490008,POSCO홀딩스,KOSPI,,403000,1,4000,1.00,402000,403000,400500,21307,8563281500,34082205690000,84571230,STK
9,035420,KR7035420009,NAVER,KOSPI,,192500,1,3700,1.96,195000,195500,191500,403539,78199907500,31263654345000,162408594,STK
10,006400,KR7006400006,삼성SDI,KOSPI,,442500,2,-1000,-0.23,443000,445000,440000,26520,11724308500,30428304525000,68764530,STK


In [5]:
kospi_data.shape

(200, 17)

이제 전날의 가격을 기준으로 매수 기준가를 만들기 위해 Range를 계산해야한다.   
   
Range는 전날 고가 - 저가임으로 전날의 가격 정보를 수집한다.   
뽑은 199개 종목을 개별로 반복문을 돌려서 가격 정보를 수집하고 마지막에 range값을 계산해주면,

In [7]:
t_date = "2024-05-02"

before_price = None
for t_i2 in kospi_data["Code"]:
    t_n = kospi_data[kospi_data["Code"]==t_i2]["Name"].values[0]
    price_data = fdr.DataReader(t_i2, t_date,t_date)
    price_data.insert(0,"Code",t_i2)
    price_data.insert(0,"Name",t_n)
    before_price = pd.concat([before_price,price_data.tail(1)],axis = 0)
before_price["range"] =  before_price["High"] - before_price["Low"]
before_price.head(5)

,Name,Code,Open,High,Low,Close,Volume,Change,range
Date,,,,,,,,,
2024-05-02,삼성전자,005930,77600,78600,77300,78000,18900640,0.006452,1300
2024-05-02,SK하이닉스,000660,169000,174700,169000,173600,3168250,-0.003444,5700
2024-05-02,LG에너지솔루션,373220,385000,392000,381000,389000,121266,0.000000,11000
2024-05-02,삼성바이오로직스,207940,777000,783000,774000,775000,49111,-0.007682,9000
2024-05-02,현대차,005380,249000,251500,246500,249000,602350,-0.007968,5000


이런 테이블이 만들어진다.   
글을 작성하는 날이 10월 25일이므로 전날인 24일 데이터를 뽑았다.   
   
이제 실시간으로 조회하기 위한 준비가 되었다.

In [9]:
#while True:
# 1번만 반복
for idx in [0]:
    for t_i in kospi_data["Code"]:
        t_n = kospi_data[kospi_data["Code"]==t_i]["Name"].values[0]
        price_data = fdr.DataReader(t_i, t_date).tail(1)
    
        price_data.insert(0,"Code",t_i)
        price_data.insert(0,"Name",t_n)

        target_band = before_price[before_price["Code"]==t_i]["range"].values[0]
        price_data["Price_stand"] = price_data["Open"] + int(target_band*1.5)
        
        base_price = price_data["Price_stand"].values[0]
        current_price = price_data["Close"].values[0]

        if current_price >= base_price:
            print(t_n, current_price, base_price,target_band)

제일기획 19410 19390 200
세아베스틸지주 22800 22775 750
제일기획 19430 19390 200


내가 구현한 코드이다.   
먼저 지속적으로 반복할 수 있게 while문을 걸어주고 모든 종목에 대해 반복하도록 for문을 넣었다.   
   
전략은 앞서 소개한 글과 동일하게 구현했다.   
   
전날 range의 1.5배가 되는 가격을 기준가로 설정해 두고,   
현재 가격이 기준 가격을 넘어설 때 시그널을 print문을 통해 종목과 현재가, 기준가, range를 출력해준다.   
   
10월 25일 1시 42분 코드를 실행해보니   
DB하이텍 43700 42375 750   
삼성전자 58300 58200 800   
두 개 기업이 나왔다.   
코드가 정상적으로 실행되고 있는 것 같다.   
   
기본적인 감지 코드는 구현이 되었다.   
   
그럼 이제 생각해 봐야 할 부분을 몇 가지 나열해보면   
1. 날짜가 바뀔 때마다 전날 데이터를 새로 수집해야 하는지
2. 주식 시장이 열릴 때 일일이 실행시켜주어야 하는지
3. 실행시켰을 때 반복적으로 같은 종목이 출력되는 부분
4. 실행 내역과 모니터링 내역의 로그 기록
5. 카카오톡 API로 나에게 메세지 보내기
6. 카카오톡 API 토큰 자동 갱신
   
정도가 될 것 같다.   
   
한 글에 담기에 내용이 많아 다음 글부터 하나하나 풀어보겠다.

# 6. 주식 단타 전략 모니터링 시스템 - 파이썬 자동 감지 프로그램(2)

앞서 말한 고려사항 중   
1. 날짜가 바뀔 때 마다 전날 데이터를 새로 수집해야 하는지
2. 주식 시장이 열릴 때 일일이 실행시켜주어야 하는지
3. 실행시켰을 때 반복적으로 같은 종목이 출력되는 부분
4. 실행 내역과 모니터링 내역의 로그 기록
5. 카카오톡 API로 나에게 메세지 보내기
6. 카카오톡 API 토큰 자동 갱신
   
밑줄 친 4개 항목에서 수정을 하겠다.   
   
그럼 필요한 패키지 두 개가 있다.

In [10]:
import datetime as dt
import time

현재 시간을 불러주는 datetime과 시간과 관련된 time 패키지를 import 한다.

In [12]:
#while True:
# 1번만 반복
# 실행전에 monitoring_log 폴더 생성
for idx in [0]:
    x = dt.datetime.now()
    file_name = "./monitoring_log/"+str(x.month) + str(x.day) +"_kospi_log"
    time_log = str(x.month)+"월"+str(x.day)+"일 "+str(x.hour%24)+":"+str(x.minute)
    if x.hour >=9 and x.hour <16:

        with open(file_name+'.txt', 'a') as f: 
            f.write(time_log + " --- Monitoring 종목 개수 : "+ str(kospi_data.shape[0])+"\n")

        set_before = True
        print(x.hour,x.minute)
        for t_i in kospi_data["Code"]:
            t_n = kospi_data[kospi_data["Code"]==t_i]["Name"].values[0]
            price_data = fdr.DataReader(t_i, t_date).tail(1)
            date = price_data.index
            if t_date != date : t_date = str(price_data.index)[16:26]
            price_data.insert(0,"Code",t_i)
            price_data.insert(0,"Name",t_n)

            target_band = before_price[before_price["Code"]==t_i]["range"].values[0]
            price_data["Price_stand"] = price_data["Open"] + int(target_band*1.5)
            base_price = price_data["Price_stand"].values[0]
            current_price = price_data["Close"].values[0]

            bot_log = (time_log + " ### 프로그램 log test ### 종목명 : " + t_n  + " / 종목코드 : " + t_i + 
                                " / 현재가격 : " + str(current_price) + " / 기준가격 : " + str(base_price) +
                                " / Range : "  +str(target_band)  +"\n")
            
            #log test
            with open(file_name+'_kospi_test.txt', 'a') as f: 
                f.write(bot_log) 
            if current_price >= base_price:
                kospi_data = kospi_data[kospi_data["Code"]!=t_i]

                print(t_n, current_price, base_price,target_band)
                with open(file_name+'.txt', 'a') as f: 
                    f.write(bot_log)

    elif x.hour == 16 and x.minute >= 10 and set_before == True:
        print(x.month, x.day,x.hour,x.minute)
        price_data = fdr.DataReader(t_i, t_date)
        t_date = str(price_data.tail(1).index)[16:26]

        kospi_data = df_krx[df_krx["Market"].str.contains("KOSPI")]
        kospi_data = kospi_data[kospi_data["Name"].isin(kospi_200_list[kospi_200_list["Code"]==102110].STK_NM_KOR)]

        before_price = None
        for t_i2 in kospi_data["Code"]:
            t_n = kospi_data[kospi_data["Code"]==t_i2]["Name"].values[0]
            price_data = fdr.DataReader(t_i2, t_date)
            price_data.insert(0,"Code",t_i2)
            price_data.insert(0,"Name",t_n)
            before_price = pd.concat([before_price,price_data.tail(1)],axis = 0)
        before_price["range"] =  before_price["High"] - before_price["Low"]
        set_before = False
        print(x.month,x.day,"completed")
        before_price.to_csv("./before_data/before_data_kospi"+str(x.month)+str(x.day)+".csv")
        with open(file_name+'.txt', 'a') as f: 
            f.write(time_log + ' --- Before_data_kospi를 갱신했습니다.'  +"\n")   

    else:
        with open(file_name+'.txt', 'a') as f: 
            f.write(time_log + ' --- 거래시간이 아닙니다. 프로그램 대기중'  +"\n")   
        time.sleep(60)

2. 주식 시장이 열릴 때 일일이 실행시켜주어야 하는지   
   
우선 dt.datatime.now() 함수는 현재 시간을 알 수 있다.   
표준시로 출력이 됨으로 한국 시간에 맞춰서 사용해야 한다.   
   
datetime으로 알 수 있는 건 년, 월, 일, 시간, 분, 초 정보이다.   
이중에 필요한 건 hour, minute이다.   
   
```python
while True:
    x = dt.datetime.now()
    time_log = str(x.month)+"월"+str(x.day)+"일 "+str(x.hour%24+9)+":"+str(x.minute)
    if x.hour+9 >=9 and x.hour+9 <16:
        print(x.hour+9,x.minute)
        
    elif x.hour+9 == 16 and x.minute >= 10 and set_before == True:
        print(x.hour+9,x.minute)
        set_before = False
    else:
        print(x.hour+9,x.minute)
        time.sleep(60)
```

datetime과 관련된 부분을 뽑아서 보면 if 문에서 사용하고 있다.   
hour에 9를 더해준 것은 한국시간에 맞추기 위해서이다.   
   
첫 번째 if 문을 보면 한국시간 기준으로 9시~16시 사이에 해당될 때, 해당 조건문이 실행이 된다.   
즉, 주식 개장 시간과 동일하다.(3시반까지지만 4시까지 거래가 가능하기에)   
   
그 다음 elif 조건은 16시 10분이 넘어가는 시점에 실행되는 조건문이다.   
여기서는 당일 주식시장이 끝난 후 주가 데이터를 수집하기 위함이다.   
따라서 한 번 실행된 후 set_before 값이 False로 바뀌며 실행되지 않는다.   
   
세 번째 else는 그 외 시간에서 실행되며 아무 작업도 하지 않는다.   
주식시장이 폐장했기 때문에 프로그램도 대기 상태로 만들기 위함이다.   
여기서 time 패키지의 sleep을 60초씩 걸어둔다.   
무의미한 루프를 계속 돌릴 필요가 없기 때문에 1분에 한 번씩만 상태 점검차 실행한다.   
   
조건문을 위와 같이 구현하면 주식 시장 시간에 맞춰 봇이 감지를 시작하고 폐장하면 봇도 대기 상태로 바뀌는 프로그램을 구현할 수 있다.   
   
비록 휴일에도 실행된다는 단점이 있지만, 휴일을 지정해주어야 하는 문제가 있기 때문에 큰 문제가 없는 한 이대로 실행해도 될 것 같다.   
   
1. 날짜가 바뀔 때마다 전날 데이터를 새로 수집해야 하는지   
   
이제 전날 데이터를 주식시장이 폐장 시 실행시켜 준비를 해놓아야 한다.   
방금의 조건문에서 elif 조건을 보면 16시 10분이 넘어가면 한 번 실행한다 했다.   
이때 다음날 사용하기 위한 데이터를 수집하는 것이다.   
   
```python
elif x.hour+9 == 16 and x.minute >= 10 and set_before == True:
    print(x.month, x.day,x.hour+9,x.minute)
    price_data = fdr.DataReader(t_i, t_date)
    t_date = str(price_data.tail(1).index)[16:26]

    kospi_data = df_krx[df_krx["Market"].str.contains("KOSPI")]
    kospi_data = kospi_data[kospi_data["Name"].isin(kospi_200_list[kospi_200_list["Symbol"]==102110].STK_NM_KOR)]

    before_price = None
    for t_i2 in kospi_data["Symbol"]:
        t_n = kospi_data[kospi_data["Symbol"]==t_i2]["Name"].values[0]
        price_data = fdr.DataReader(t_i2, t_date)
        price_data.insert(0,"Symbol",t_i2)
        price_data.insert(0,"Name",t_n)
        before_price = pd.concat([before_price,price_data.tail(1)],axis = 0)
    before_price["range"] =  before_price["High"] - before_price["Low"]
    set_before = False
    print(x.month,x.day,"completed")
```
    
해당 조건문만 따로 빼내 와서 보자.   
   
현재 일 기준으로 데이터를 수집해야 하기 때문에 아무 종목이나 데이터를 한번 수집하고, 가장 최근 날짜로 업데이트해준다.   
   
```python
price_data = fdr.DataReader(t_i, t_date)
t_date = str(price_data.tail(1).index)[16:26]
```

.tail(1)을 사용한 이유가 가장 최신 데이터가 맨 아래에 있기 때문이다. 여기서 문자열의 위치를 지정해 날짜만 추출한다.   
   
그 다음 대상 종목을 재지정해주고, 전날 가격으로 사용할 데이터를 수집한다.   
해당 코드는 이전 글에 설명했던 코드와 동일하다.   
2022.10.25 - [개발기록] - 주식 단타 전략 모니터링 시스템 개발 일지 - 5. 자동 감지 프로그램(1)   
혹시 모를 에러에 대비해 해당 데이터를 저장한다.   
   
그리고 set_before값을 False로 바꿔 elif문이 다시 실행되지 않도록 바꾼다.   
set_before값은 다음날이 되어 주식 시장이 개장하면 다시 True로 바뀌게 설정해놓았다.   
   
이렇게 전날 가격 데이터도 자동으로 수집할 수 있게 구현했다.   
   
3. 실행시켰을 때 반복적으로 같은 종목이 출력되는 부분   
같은 종목이 반복적으로 출력되지 않게 하는 방법은 간단하다.   
   
```python
if current_price >= buy_price :
    kospi_data = kospi_data[kospi_data["Symbol"]!=t_i]
    print(t_n, current_price, buy_price,target_band)
```

현재 가격이 기준이 매수 기준 가격보다 높아지면 print문으로 출력하도록 조건을 걸어놓았는데, 여기서 출력되는 종목을 감지 대상 종목 리스트에서 제외하면 된다.   
   
4. 실행 내역과 모니터링 내역의 로그 기록   
프로그램이 잘 돌아가는지 로그를 남기는 것도 중요하다.   
출력되는 결과물을 일 별로 저장하면서 오류 여부를 찾아야 하기 때문이다.   
해당 부분 코드만 뽑아서 보면   
   
```python
while True:
    x = dt.datetime.now()
    file_name = "./monitoring_log/"+str(x.month) + str(x.day) +"_kospi_log"
    time_log = str(x.month)+"월"+str(x.day)+"일 "+str(x.hour%24+9)+":"+str(x.minute)
    if x.hour+9 >=9 and x.hour+9 <16:

        with open(file_name+'.txt', 'a') as f: 
            f.write(time_log + " --- Monitoring 종목 개수 : "+ str(kospi_data.shape[0])+"\n")

        for t_i in kospi_data["Symbol"]:

            bot_log = (time_log + " ### 프로그램 log test ### 종목명 : " + t_n  + " / 종목코드 : " + t_i + 
                                " / 현재가격 : " + str(current_price) + " / 기준가격 : " + str(base_price) +
                                " / Range : "  +str(target_band)  +"\n")
            
            with open(file_name+'_test.txt', 'a') as f: 
                f.write(bot_log) 
            if current_price >= base_price:
                kospi_data = kospi_data[kospi_data["Symbol"]!=t_i]
                print(t_n, current_price, base_price,target_band)
                with open(file_name+'.txt', 'a') as f: 
                    f.write(bot_log)

    elif x.hour+9 == 16 and x.minute >= 10 and set_before == True:
        
        with open(file_name+'.txt', 'a') as f: 
            f.write(time_log + ' --- Before_data_kospi를 갱신했습니다.'  +"\n")   

    else:
        with open(file_name+'.txt', 'a') as f: 
            f.write(time_log + ' --- 거래시간이 아닙니다. 프로그램 대기중'  +"\n")   
        time.sleep(60)
```

file_name의 경우 날짜 별로 저장하기 위해 datetime에서 month와 day를 사용해 양식을 만들었다.   
time_log의 경우 저장하는 시간을 체크하기 위해 월, 일, 시간, 분을 조합해 만들었다.   
   
생성되는 파일은 두 개다.   
프로그램이 잘 실행되고 감지되는 종목을 기록하는 "filename".txt 파일과   
모니터링이 잘 되고 있는지 모든 종목을 기록하는 "filename_test".txt파일이다.   
   
전체 감지 대상 종목이 200개라 할 때, 200개를 한 번 볼 때마다 몇 개가 감지되는지 종목 개수를 출력하다.   
개수를 출력하는 이유는 매수 사인이 난 종목을 출력 후 해당 종목을 감지 대상에서 제외하는데 그럼 199개가 감지되고 있는지 확인하기 위함이다.   
   
bot_log에 모니터링하는 종목, 가격 등을 기록한 후   
with open (file_name.... 부터 시작하는 코드에서 각각 해당 파일에 기록한다.   
전날 가격 데이터를 생성하거나, 거래시간이 아닐 때도 기록해 프로그램이 잘 실행되고 있는지 체크하도록 하였다.   
   
프로그램을 실행시켜 놓으면 날짜가 바뀌며 자동으로 저장되도록 구현되어 있다.   
   
한 가지, 표준시간 +9로 사용하고 있기 때문에 날짜도 9시간 후에 바뀐다.   
즉, 주식 시장이 개장하면 표준 날짜가 바뀌기 때문에 9시부터 해당 거래일을 로그가 저장된다.   
나는 이게 보기 편해서 그대로 사용하고 있다.   
   
작업했던 결과를 보면   
2개 종목이 감지되어 추천한 후 폐장시간까지 기준가를 넘는 종목이 없어 모니터링만 지속한다.   
16시가 넘는 시점부터 잠시 대기하다가 16시 10분 시점에 전날 가격으로 사용할 데이터를 갱신한다.   
그 후 다음 거래 시작시간까지 대기상태에 돌입한다.   
   
여기까지 주요 내용에 대해선 구현이 되었다.   
   
이제 남은 것은   

5. 카카오톡 API로 나에게 메세지 보내기
6. 카카오톡 API 토큰 자동 갱신

두 항목인데..   
   
글이 너무 길어져 다음 글에서 다루겠다.   
   
카카오톡 API까지 구현하고 나면 생각했던 대부분 기능을 구현하는 것 같다.   
   
해당 내용을 소개한 후 앞으로 모니터링 시스템을 어떻게 발전시킬지 정리해보겠다.

# 7. 주식 단타 전략 모니터링 시스템 - 파이썬 자동 감지 프로그램(3)

지난 글에 이어 카카오톡 내게 메세지 보내기를 구현하는 것으로 1차 개발을 마무리하고자 한다.   
   
앞선 내용를 정리하면   
전날 가격을 이용해 range를 계산하고, 이 값으로 매수 기준가를 설정 한 다음, 당일 주식 가격이 기준가를 넘는 순간 매수 시그널을 보내는 것까지 구현했다.   
시그널은 현재 Jupyter notebook의 셀 출력 값과 로그파일을 통해서 기록되기 때문에 직접 모니터를 보고 있어야지만 시그널을 보낸 것을 알 수 있었다.   
이 때문에 다른 일을 하고 있어도 매수 시그널을 알려줄 무언가가 필요했다.   
   
이런 이유로 가장 먼저 생각난 것이 카카오톡이다.   
매수 시그널이 뜰 때 마다 메세지를 보내 알려줄 수 있다면, 다른 일을 하면서도 체크를 할 수 있겠다 생각했다.   
   
로직을 간단히 설명하면   
매수 기준가를 이용해 한 종목씩 모니터링를 진행하다 매수 기준가를 넘으면 해당 종목에 대한 정보를 카카오톡 API를 이용해 메세지를 보낸다.   
이때 담고 있는 정보는 종목, 가격, 매수기준가, Range 등의 정보와 해당 종목을 자세하기 확인하기 위해 링크를 네이버 금융의 해당 종목 페이지를 링크로 보낸다.   
한 가지, 카카오톡 API의 토큰은 만료기간이 있으므로 메세지 보내기를 실패했을 때 자동 갱신 후 다시 보내는 로직을 심어야 했다.   
그리고, 자동감지 프로그램은 24시간 내내 돌아가도록 설계하였기 때문에 중간에 오류가 나서 멈춘다면 이 역시 메세지를 보내 대응할 수 있도록 만들어야 했다. 여기서도 토큰이 만료되면 오류 메세지를 못 보내기 때문에 토큰을 갱신하는 코드를 심었다.   
   
그전에, 파이썬 자동 감지 프로그램(2)의 코드를 돌려놓으면 아침 9시에 에러가 발생해 더 이상 작동하지 않는 상황이 가끔 발생했다.   
원인을 찾아보니까 주식 데이터를 불러오는 과정에서 9시에 개장을 했지만, 아직 거래가 이뤄지지 않거나, 업데이트가 늦는 경우에 당일 가격을 불러오지 못해 빈 데이터 프레임이 생성이 되었다.   
항상 가격이 조회될 것이란 생각으로 구현했더니, 빈 프레임에 대응이 안 되었던 것이다.   
따라서 아직 가격이 조회되지 않은 종목은 넘기고 가격이 생성되어 조회되는 종목부터 모니터링하도록 코드를 수정했다.   
아래의 코드가 모든 내용이 반영된 최종 버전의 코드이다.   
   
우선 저번 코드에 이어 필요한 패키지와, 토큰을 불러와아한다.

In [15]:
import requests
import json

with open("kakao_code.json", "r") as fp:
    tokens = json.load(fp)
    
url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

headers = {
    "Authorization": "Bearer " + tokens["access_token"]
}


def refreshToken(refresh_token):
    REST_API_KEY = "aa96073b402b59689b2e3bb12c1390dc"
    REDIRECT_URI = "https://www.naver.com"

    data = {
        "grant_type": "refresh_token",
        "client_id":f"{REST_API_KEY}",
        "refresh_token": refresh_token 
    }    
 
    resp = requests.post(REDIRECT_URI , data=data)
    print(resp)
    new_token = resp.json()

    return new_token['access_token']

토큰을 재발행해주는 함수까지 넣어놓으면 준비가 끝난다.

In [21]:
try:
    #while True:
    # 1번만 반복
    for idx in [0]:
        x = dt.datetime.now()
        file_name = "./monitoring_log/"+str(x.month) + str(x.day) +"_kospi_log"
        time_log = str(x.month)+"월"+str(x.day)+"일 "+str(x.hour%24)+":"+str(x.minute)
        if x.hour >=9 and x.hour <16:

            with open(file_name+'.txt', 'a') as f: 
                f.write(time_log + " --- Monitoring 종목 개수 : "+ str(kospi_data.shape[0])+"\n")

            set_before = True
            print(x.hour,x.minute)
            for t_i in kospi_data["Code"]:
                t_n = kospi_data[kospi_data["Code"]==t_i]["Name"].values[0]
                price_data = fdr.DataReader(t_i, t_date).tail(1)
                if price_data.shape[0]!= 0:
                    date = price_data.index
                    if t_date != date : t_date = str(price_data.index)[16:26]
                    price_data.insert(0,"Code",t_i)
                    price_data.insert(0,"Name",t_n)

                    target_band = before_price[before_price["Code"]==t_i]["range"].values[0]
                    price_data["Price_stand"] = price_data["Open"] + int(target_band*1.5)
                    base_price = price_data["Price_stand"].values[0]
                    current_price = price_data["Close"].values[0]

                    bot_log = (time_log + " ### 프로그램 log test ### 종목명 : " + t_n  + " / 종목코드 : " + t_i + 
                                " / 현재가격 : " + str(current_price) + " / 기준가격 : " + str(base_price) +
                                " / Range : "  +str(target_band)  +"\n")
                    #log test
                    with open(file_name+'_test.txt', 'a') as f: 
                        f.write(bot_log) 

                    if current_price >= base_price :
                        kospi_data = kospi_data[kospi_data["Code"]!=t_i]
                        data = {"template_object" : json.dumps({ 
                                     "object_type" : "text",
                                     "text" : "매수사인"+"\n" + 
                                              "종목 : "+t_n +"\n" + "코드 : "+t_i +"\n"+ "매수가격 : "+str(current_price) +"\n"+
                                              "배수 : "+str(np.round((current_price/base_price-1)*100,2)) +"\n"+
                                              "Stand : "+ str(base_price)+ "\n"+
                                              "Range : "+ str(target_band),

                                    "link" : {"web_url": "https://finance.naver.com/item/main.naver?code="+str(t_i),
                                              "mobile_web_url":"https://finance.naver.com/item/main.naver?code="+str(t_i)}
                                })}


                        response = requests.post(url, headers=headers, data=data)
                        if response.json().get('result_code') == 0:
                            print('메시지를 성공적으로 보냈습니다.')
                            print(t_n, current_price, base_price,target_band)
                            with open(file_name+'.txt', 'a') as f: 
                                f.write(time_log + ' --- 메시지를 성공적으로 보냈습니다.'  +"\n")
                                f.write(bot_log)
                        else:
                            tokens["access_token"] = refreshToken(tokens["refresh_token"])
                            headers = {"Authorization": "Bearer " + tokens["access_token"]}
                            response = requests.post(url, headers=headers, data=data)
                            if response.json().get('result_code') == 0:
                                print("토큰 갱신에 성공했습니다.")
                                print('메시지를 성공적으로 보냈습니다.')
                                print(t_n, current_price, base_price, target_band)
                                with open(file_name+'.txt', 'a') as f: 
                                    f.write(time_log + ' --- 토큰을 갱신했습니다.'  +"\n")
                                    f.write(time_log + ' --- 메시지를 성공적으로 보냈습니다.'  +"\n")
                                    f.write(bot_log)
                            else:
                                print("토큰 갱신에 실패했습니다.")
                                with open(file_name+'.txt', 'a') as f: f.write(time_log + ' --- 토큰 갱신에 실패했습니다.'  +"\n")                        

        elif x.hour == 16 and x.minute >= 10 and set_before == True:
            print(x.month, x.day,x.hour,x.minute)
            price_data = fdr.DataReader(t_i, t_date)
            date = str(price_data.tail(1).index)[16:26]
            t_date = date

            kospi_data = df_krx[df_krx["Market"].str.contains("KOSPI")]

            kospi_data = kospi_data[kospi_data["Name"].isin(kospi_200_list[kospi_200_list["Code"]==102110].STK_NM_KOR)]

            before_price = None
            for t_i2 in kospi_data["Code"]:
                t_n = kospi_data[kospi_data["Code"]==t_i2]["Name"].values[0]
                price_data = fdr.DataReader(t_i2, t_date)
                price_data.insert(0,"Code",t_i2)
                price_data.insert(0,"Name",t_n)
                before_price = pd.concat([before_price,price_data.tail(1)],axis = 0)
            before_price["band"] =  before_price["High"] - before_price["Low"]
            set_before = False
            print(x.month,x.day,"completed")
            before_price.to_csv("./before_data/before_data_kospi"+str(x.month)+str(x.day)+".csv")
            with open(file_name+'.txt', 'a') as f: f.write(time_log + ' --- Before_data를 갱신했습니다.'  +"\n")   

        else:
            with open(file_name+'.txt', 'a') as f: f.write(time_log + ' --- 거래시간이 아닙니다. 프로그램 대기중'  +"\n")   
            time.sleep(60)
except:
    print("Error")
    with open(file_name+'.txt', 'a') as f: f.write(time_log + ' --- 코드 에러.'  +"\n")   
    data = {"template_object" : json.dumps({ 
                 "object_type" : "text",
                 "text" : "코드에러"+"\n" ,
                 "link" : None
            })}
    response = requests.post(url, headers=headers, data=data)
    if response.json().get('result_code') == 0:
        with open(file_name+'.txt', 'a') as f: f.write(time_log + ' --- 코드 에러 메시지를 성공적으로 보냈습니다.'  +"\n")   
    else:
        tokens["access_token"] = refreshToken(tokens["refresh_token"])
        headers = {"Authorization": "Bearer " + tokens["access_token"]}
        response = requests.post(url, headers=headers, data=data)
        if response.json().get('result_code') == 0:
            with open(file_name+'.txt', 'a') as f: f.write(time_log + ' --- 코드 에러 토큰 갱신에 성공했습니다.'  +"\n")   
        else:
            with open(file_name+'.txt', 'a') as f: f.write(time_log + ' --- 코드 에러 토큰 갱신에 실패했습니다.'  +"\n")

10 40


전체 코드인데, 기능을 하나씩 추가한 것이라 조금 중복적인 내용이 많다.   
일단은 잘 돌아가니까.. 사용해보고 하나씩 정리해보겠다.   
   
가장 먼저 try, except문이 보인다.   
종목 모니터링 중 오류가 나면 except문으로 빠져서 코드 에러 메세지를 보낸다.   
만약 여기서 메세지 보내기를 실패했다면, 토큰이 만료된 것이기 때문에, 토큰 재발행을 한 후 다시 갱신되었다는 메세지를 보낸다.   
따라서 코드 에러 메세지를 받으면 바로 대응할 수 있도록 했다.   
   
종목 감지 로직은 저번 글과 동일함으로 메세지 보내기 코드 위주로 설명하자면,   

```python
data = {"template_object" : json.dumps({ 
             "object_type" : "text",
             "text" : "매수사인"+"\n" + 
                      "종목 : "+t_n +"\n" + "코드 : "+t_i +"\n"+ "매수가격 : "+str(current_price) +"\n"+
                      "배수 : "+str(np.round((current_price/base_price-1)*100,2)) +"\n"+
                      "Stand : "+ str(base_price)+ "\n"+
                      "Range : "+ str(target_band),

            "link" : {"web_url": "https://finance.naver.com/item/main.naver?code="+str(t_i),
                      "mobile_web_url":"https://finance.naver.com/item/main.naver?code="+str(t_i)}
        })}

response = requests.post(url, headers=headers, data=data)
if response.json().get('result_code') == 0:
    print('메시지를 성공적으로 보냈습니다.')
    print(t_n, current_price, base_price,target_band)
else:
    tokens["access_token"] = refreshToken(tokens["refresh_token"])
    headers = {"Authorization": "Bearer " + tokens["access_token"]}
    response = requests.post(url, headers=headers, data=data)
    if response.json().get('result_code') == 0:
        print("토큰 갱신에 성공했습니다.")
        print('메시지를 성공적으로 보냈습니다.')
        print(t_n, current_price, base_price, target_band)
    else:
        print("토큰 갱신에 실패했습니다.")
```

우선 data부분이 카카오톡 API로 메세지를 보내기 위한 자료를 만는 것이다.   
text 부분에 내가 필요한 정보인 종목, 코드, 가격 등을 저장한다.(필요한 정보에 따라 수정하면 된다.)   
link 부분은 제외하려고 했다가 메세지 보내기에 기본적으로 들어가 있어서 어떻게 활용할까 고민하다가 종목별로 네이버금융 페이지를 연결했다.   
현재 가격이 기준 가격을 넘어서 매수 시그널을 보내면, 바로 매수할 수도 있지만, 어떤 종목인지 상세히 보고 매수를 결정하기도 해서 종목코드를 이용해 링크를 만들어 보낸다.   
따라서 카카오톡 메세지를 받고 링크를 누르면 바로 정보 페이지와 바로 연결되기 때문에 검색하고 찾아가는 시간을 아낄 수 있었다.   
   
전송할 자료를 만들고 난 후는 앞서 정리했던 메세지 보내기 코드이다.   
만약 실패했다면, 토큰이 만료된 것이기 때문에 갱신 후 메세지를 다시보낸다. 그런데 두 번째 메세지 보내기도 실패한 경우는 토큰 갱신에 실패했다는 것이다.   
토큰은 유효기간뿐만 아니라 재발행할 수 있는 기간도 제한이 되어있다. 문서상 몇 개월 정도 사용할 수 있다고 한 것 같은데 이게 만료되면 토큰 자체를 새로 발행받아아 한다.   
이런 경우 번거롭지만, 카카오톡 developers에서 새로 발행받아 REST API KEY를 변경해주어야 한다.   
   
이렇게 링크를 타고 해당 종목의 상세정보를 확인 할 수 있다.   
   
이것으로 감지 프로그램의 구현을 끝냈다.   
초기에 생각했던 내용을 대부분 구현이 되었기 때문에, 단타 전략 프로그램 구현에 대한 글은 여기까지 쓰도록 하겠다.    
   
이제 실제로 내가 사용해 볼 차례인데.. 어떤 종목을 대상으로 할지 정해야 할 것 같다.   
코스피 200개 기업을 대상으로 하면 모든 종목을 한 번 다 보는데 약 100초가 걸린다.   
빠르게 봐야 하는 경우 종목을 조절해야 할 것이다.   
   
그리고 실제로 써먹기 위해선 몇 가지 수정이 필요하다 생각된다.   
   
단타 전략 수정 - 전날 가격 기준 range를 이용해 매수기준을 잡는데 좀 더 디테일하게 설정이 필요할 것 같다.   
매수 타이밍 선정 - 매수 시그널이 떴을 때 바로 사는 게 좋은지, 종가에 사는게 좋은지 테스트가 필요하다.   
카카오톡 소리 알림 기능 없음 - 메세지가 와도 소리 알림이 없으니 pc카톡으로 한쪽 구석에 띄워 놓거나 해야 한다. 더 좋은 알림 방법을 찾아야 할 것 같다.   
전략이 수정되면 증권 API와 연결해 자동 매매가 이뤄지게 바꿀 예정이다. 아무래도 사람의 손으로 하나하나 매수, 매도를 하는 것을 번거롭기 때문이다.   
   
위의 내용은 당장 계획을 하고 있지 않기 때문에 테스트 혹은 구현할 때마다 추가 글로 소개를 하겠다.   
   
그리고 장기적으론 단타 전략이 아닌 종목 분석 전략도 하나씩 구현 혹은 만들어 보려 한다.   
   
딥러닝을 이용해서 주가를 예측하는 논문이 많이 나오고 있다.   
시간이 허락한다면, 몇 가지를 소개하는 글을 써보도록 하고, 추후엔 직접 구현하는 것까지 해보겠다.   
   
이상 주식 단타 전략 모니터링 시스템 개발 일지를 마친다.